# Design Pattern - Template Method

> Wikipedia: In software engineering, the template method pattern is a behavioral design pattern that **defines the program skeleton of an algorithm in an operation**, **deferring some steps to subclasses**. It lets one redefine certain steps of an algorithm without changing the algorithm's structure.

1. 父類別只定義 methods 的框架但不實作。
2. 子類別中以不修改框架的原則下實作 methods。

In [3]:
#呼叫 abc library, 解決 Python 沒有抽象類別（abstract class）的問題。
import abc

class MachineLearningPipeline(object):
    """
    用abc定義抽象化的機器學習流程父類別，並定義流程執行順序。
    父類別的 method 可以自行決定 parameter 的定義，
    這裏的範例限制 get_sourcedata method 的 parameter 只能輸入 data 參數，
    好處是可以控制子類別的彈性。
    其他 methods 則允許任何參數的輸入，
    好處是當子類別實作同一個 methods 但需要傳不同參數時仍可以調用。
    """
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def get_sourcedata(self, data):
        pass
    
    @abc.abstractmethod
    def preprocess_data(self, *args, **kwargs):
        pass
    
    @abc.abstractmethod
    def engineer_feature(self, *args, **kwargs):
        pass
    
    @abc.abstractmethod
    def fit_model(self, *args, **kwargs):
        pass
    
    @abc.abstractmethod
    def evaluate_result(self, *args, **kwargs):
        pass
    
    # Hook method
    def sample_data(self, *args, **kwargs):
        pass
    
    #定義method的執行流程
    def exexute_pipeline(self, data):
        self.get_sourcedata(data)
        self.sample_data()
        self.preprocess_data()
        self.engineer_feature()
        self.fit_model()
        self.evaluate_result()
        

class BinaryClassification(MachineLearningPipeline):
    """
    實作一個二元分類的類別，實作父類別抽象後的 methods
    """
    def __init__(self):
        self.pipeline = ""
    
    def get_sourcedata(self, data):
        self.pipeline += '資料源：' + str(data) + '\n'
        return self
    
    def sample_data(self):
        self.pipeline += '抽樣：' + 'underSampling' + '\n'
        return self
    
    def preprocess_data(self):
        self.pipeline += '前處理：' + 'stringIndexer, onehotEncoder, vectorAssembler' + '\n'
        return self
    
    def engineer_feature(self):
        self.pipeline += '特徵工程：' + 'mathTransform, dimensionReduction' + '\n'
        return self

    def fit_model(self):
        self.pipeline += '模型：' + 'SVM' + '\n'
        return self
    
    def evaluate_result(self):
        self.pipeline += '結果評估：' + 'Recall, Precision, AUC' + '\n'
        return self
    
    
class MulticlassCalssification(MachineLearningPipeline):
    """
    實作一個多元分類的類別，同樣實作父類別抽象後的 methods
    """
    def __init__(self):
        self.pipeline = ""
    
    def get_sourcedata(self, data):
        self.pipeline += '資料源：' + str(data) + '\n'
        return self
    
    def preprocess_data(self):
        self.pipeline += '前處理：' + 'stringIndexer, labelEncoder, vectorAssembler' + '\n'
        return self
    
    def engineer_feature(self):
        self.pipeline += '特徵工程：' + 'mathTransform, dimensionReduction' + '\n'
        return self

    def fit_model(self):
        self.pipeline += '模型：' + 'SVM, OneVsRestModel' + '\n'
        return self

    def evaluate_result(self):
        self.pipeline += '結果評估：' + 'Recall, Precision, F1-Score' + '\n'
        return self    
        
if __name__ == '__main__':
    bin_class = BinaryClassification()
    multi_class = MulticlassCalssification()
    bin_data = ['vp_bank.event_cc_txn', 'vp_bank.chp_event_cust']
    multi_data = ['vp_bank.event_cc_txn', 'vp_bank.chp_event_cust', 'vp_bank.acct_cc_daily']
    bin_class.exexute_pipeline(bin_data)
    multi_class.exexute_pipeline(multi_data)
    print('=====start BinaryClassification pipeline=====')
    print('Binary Classifier pipeline：\n', bin_class.pipeline)
    print('=====start MulticlassCalssification pipeline=====')
    print('Multiclass Calssifier pipeline：\n', multi_class.pipeline)
    print('=====end of pipeline=====')

=====start BinaryClassification pipeline=====
Binary Classifier pipeline：
 資料源：['vp_bank.event_cc_txn', 'vp_bank.chp_event_cust']
抽樣：underSampling
前處理：stringIndexer, onehotEncoder, vectorAssembler
特徵工程：mathTransform, dimensionReduction
模型：SVM
結果評估：Recall, Precision, AUC

=====start MulticlassCalssification pipeline=====
Multiclass Calssifier pipeline：
 資料源：['vp_bank.event_cc_txn', 'vp_bank.chp_event_cust', 'vp_bank.acct_cc_daily']
前處理：stringIndexer, labelEncoder, vectorAssembler
特徵工程：mathTransform, dimensionReduction
模型：SVM, OneVsRestModel
結果評估：Recall, Precision, F1-Score

=====end of pipeline=====


# Tip: 子類別的流程有些許不同時該如何處理？

### Hook method
可以在父類別增加供 override 的 hook method, 子類別之間的實作流程不一定完全一樣，此時可以使用 hook 的方式微調流程。 

# 解決了什麼問題

透過將通用算法抽象出來，解決一些方法通用，卻在每一個子類都重寫了這一方法。

# 適用性

1. 當你只希望客戶端擴展某個特定步驟，而不是整個程式流程或其結構時，可使用template method pattern。
2. Template method 將整個流程轉換為一系列獨立的步驟，以便子類別能對其進行擴展，同時還可讓抽象化父類別中所定義的結構保持完整。
3. 當多個類別的流程除一些細微不同之外幾乎完全一樣時，你可使用該模式。但其後果就是，只要算法發生變化，你就可能需要修改所有的類。
4. 在將算法轉換為 template method 時，你可將相似的實現步驟提取到父類別中以去除重複代碼。子類別間各自不同的程式碼可繼續保留在子類別中。

# 資料科學的相關場景

1. 自己認為滿適合定義 ml pipeline 的藍圖，例如 Feature Engineering 裡面**類別變數**和**數值變數**雖然屬性不同，但可能都會有一些操作流程類似，也可以抽象出來再到子類別實作。
2. 我認為跟 Tensorflow 1.x版的 placeholder 有點像，先宣告空的 tensor 但沒有任何動作，後續執行階段才會餵資料進去開始執行。

# 與其他design pattern的比較

1. 之前介紹過的 Factory method 可以視為是 template method 的一個特殊形式。 (Head First Design Patterns book)